## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


The cell below contain the library imports needed to read the CSV data files for The Numbers and Rotten Tomatoes.

In [1]:
# Your code here - remember to use markdown cells for comments as well!
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Testing to see if we can bring and API in to supplement our data
import requests
import json
url = 'https://api.themoviedb.org/3/movie/{movie_id}?api_key=<<503f12af247746f5fc6113f4fed1f3a3>>&language=en-US'
TMDB = requests.get(url)

In [3]:
type(TMDB)

requests.models.Response

In [4]:
# Attemp to access an API
TMDB.status_code

401

In [5]:
TMDB_data = TMDB.json()

In [7]:
TMDB_data['title']

KeyError: 'title'

In [8]:
#Unzipping the SQL
from zipfile import ZipFile
# specifying the zip file name
file_name = "./zippedData/im.db.zip"
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

File Name                                             Modified             Size
im.db                                          2021-12-20 16:31:38    169443328
Extracting all the files now...
Done!


In [9]:
#Establish the SQL connection
conn = sqlite3.connect('im.db')

In [10]:
# FILES WE WILL USE FOR OUR ANALYSIS
TN_DF = pd.read_csv("./zippedData/tn.movie_budgets.csv.gz")
ROT_TOM_INFO_DF = pd.read_csv("./zippedData/rt.movie_info.tsv.gz", sep="\t")
    
# FILES THAT WE ARE NOT GOING TO USE IN OUR ANALYSIS
ROT_TOM_REV_DF = pd.read_csv("./zippedData/rt.reviews.tsv.gz", sep="\t", encoding='windows-1252')
TMDB_DF = pd.read_csv("./zippedData/tmdb.movies.csv.gz")
BOM_DF = pd.read_csv("./zippedData/bom.movie_gross.csv.gz")

These are the tables that have the information we are going to combine.

In [11]:
cur = conn.cursor()

In [12]:
cur.execute("""SELECT name FROM sqlite_master WHERE type = 'table';""")
# Fetch the result and store it in table_names
table_names = cur.fetchall()
table_names

[('movie_basics',),
 ('directors',),
 ('known_for',),
 ('movie_akas',),
 ('movie_ratings',),
 ('persons',),
 ('principals',),
 ('writers',)]

In [13]:
IMDB_ratings = pd.DataFrame(
    data=cur.execute("""SELECT * FROM movie_ratings;""").fetchall(),
    columns=[x[0] for x in cur.description]
)
IMDB_ratings.shape

(73856, 3)

In [14]:
#This sets the var sql = to the data frame
IMDB = pd.DataFrame(
    data=cur.execute("""SELECT * FROM movie_basics;""").fetchall(),
    columns=[x[0] for x in cur.description]
)

In [15]:
IMDB['start_year'].min()

2010

In [16]:
IMDB_clean = IMDB.copy()

In [17]:
IMDB_clean = IMDB_clean[IMDB['start_year'] >= 2010]

In [18]:
IMDB_clean.shape

(146144, 6)

In [19]:
#I Looked it uo the world wide numbers are actually the total numbers so Im renaming that colomn
TN_DF

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


Below we are cleaning the data by removing the ID column and also the $ sign and commas from the box office numbers. We also are converting the box offices numbers to int in order to make a new column to represent net income.

In [20]:
TN_clean = TN_DF.copy()

In [21]:
TN_clean.drop('id', axis=1, inplace=True)

In [22]:
TN_clean

,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...
5777,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [23]:
# Stripping $

TN_clean['production_budget'] = TN_clean['production_budget'].map(lambda x: x.strip('$,'))
TN_clean['domestic_gross'] = TN_clean['domestic_gross'].map(lambda x: x.strip('$,'))
TN_clean['worldwide_gross'] = TN_clean['worldwide_gross'].map(lambda x: x.strip('$,'))

In [24]:
TN_clean

,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,"Dec 18, 2009",Avatar,"425,000,000","760,507,625","2,776,345,279"
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"410,600,000","241,063,875","1,045,663,875"
2,"Jun 7, 2019",Dark Phoenix,"350,000,000","42,762,350","149,762,350"
3,"May 1, 2015",Avengers: Age of Ultron,"330,600,000","459,005,868","1,403,013,963"
4,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"317,000,000","620,181,382","1,316,721,747"
...,...,...,...,...,...
5777,"Dec 31, 2018",Red 11,"7,000",0,0
5778,"Apr 2, 1999",Following,"6,000","48,482","240,495"
5779,"Jul 13, 2005",Return to the Land of Wonders,"5,000","1,338","1,338"
5780,"Sep 29, 2015",A Plague So Pleasant,"1,400",0,0


In [25]:
TN_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   release_date       5782 non-null   object
 1   movie              5782 non-null   object
 2   production_budget  5782 non-null   object
 3   domestic_gross     5782 non-null   object
 4   worldwide_gross    5782 non-null   object
dtypes: object(5)
memory usage: 226.0+ KB


In [26]:
# stripping ',''

TN_clean = TN_clean.apply(lambda x: x.str.replace(',', ''))

In [27]:
TN_clean

,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,Dec 18 2009,Avatar,425000000,760507625,2776345279
1,May 20 2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,Jun 7 2019,Dark Phoenix,350000000,42762350,149762350
3,May 1 2015,Avengers: Age of Ultron,330600000,459005868,1403013963
4,Dec 15 2017,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747
...,...,...,...,...,...
5777,Dec 31 2018,Red 11,7000,0,0
5778,Apr 2 1999,Following,6000,48482,240495
5779,Jul 13 2005,Return to the Land of Wonders,5000,1338,1338
5780,Sep 29 2015,A Plague So Pleasant,1400,0,0


In [28]:
TN_clean[['production_budget', 'domestic_gross', 'worldwide_gross']] = TN_clean[['production_budget', 'domestic_gross', 'worldwide_gross']].apply(pd.to_numeric)

In [29]:
TN_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   release_date       5782 non-null   object
 1   movie              5782 non-null   object
 2   production_budget  5782 non-null   int64 
 3   domestic_gross     5782 non-null   int64 
 4   worldwide_gross    5782 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 226.0+ KB


We are adding in profitability metrics to the TN_clean data set, including: 1) Net Income 2) Net Income Ratio and 3) ROI.

In [30]:
TN_clean['Net Income'] = TN_clean['worldwide_gross'] - TN_clean['production_budget']

In [82]:
TN_clean['Net_Income_Ratio'] = TN_clean['Net Income'] / TN_clean['worldwide_gross']

In [83]:
TN_clean['ROI'] = TN_clean['Net Income'] / TN_clean['production_budget']

In [84]:
TN_clean

,release_date,movie,production_budget,domestic_gross,worldwide_gross,Net Income,release_year,Net_Income_Ratio,ROI
0,2009-12-18,Avatar,425000000,760507625,2776345279,2351345279,2009,0.846921,5.532577
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2011,0.607331,1.546673
2,2019-06-07,Dark Phoenix,350000000,42762350,149762350,-200237650,2019,-1.337036,-0.572108
3,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963,1072413963,2015,0.764364,3.243841
4,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,999721747,2017,0.759251,3.153696
...,...,...,...,...,...,...,...,...,...
5777,2018-12-31,Red 11,7000,0,0,-7000,2018,-inf,-1.000000
5778,1999-04-02,Following,6000,48482,240495,234495,1999,0.975051,39.082500
5779,2005-07-13,Return to the Land of Wonders,5000,1338,1338,-3662,2005,-2.736921,-0.732400
5780,2015-09-29,A Plague So Pleasant,1400,0,0,-1400,2015,-inf,-1.000000


In [85]:
TN_clean['release_date'] = pd.to_datetime(TN_clean['release_date'])

In [86]:
TN_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   release_date       5782 non-null   datetime64[ns]
 1   movie              5782 non-null   object        
 2   production_budget  5782 non-null   int64         
 3   domestic_gross     5782 non-null   int64         
 4   worldwide_gross    5782 non-null   int64         
 5   Net Income         5782 non-null   int64         
 6   release_year       5782 non-null   int64         
 7   Net_Income_Ratio   5782 non-null   float64       
 8   ROI                5782 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(5), object(1)
memory usage: 406.7+ KB


In [87]:
TN_clean.sort_values(by=['Net Income'], ascending=False)

,release_date,movie,production_budget,domestic_gross,worldwide_gross,Net Income,release_year,Net_Income_Ratio,ROI
0,2009-12-18,Avatar,425000000,760507625,2776345279,2351345279,2009,0.846921,5.532577
42,1997-12-19,Titanic,200000000,659363944,2208208395,2008208395,1997,0.909429,10.041042
6,2018-04-27,Avengers: Infinity War,300000000,678815482,2048134200,1748134200,2018,0.853525,5.827114
5,2015-12-18,Star Wars Ep. VII: The Force Awakens,306000000,936662225,2053311220,1747311220,2015,0.850972,5.710167
33,2015-06-12,Jurassic World,215000000,652270625,1648854864,1433854864,2015,0.869606,6.669092
...,...,...,...,...,...,...,...,...,...
352,2001-04-27,Town & Country,105000000,6712451,10364769,-94635231,2001,-9.130472,-0.901288
341,2019-06-14,Men in Black: International,110000000,3100000,3100000,-106900000,2019,-34.483871,-0.971818
193,2011-03-11,Mars Needs Moms,150000000,21392758,39549758,-110450242,2011,-2.792691,-0.736335
194,2020-12-31,Moonfall,150000000,0,0,-150000000,2020,-inf,-1.000000


Next, we are cleaning the IMDB table to remove "N/A" or "Null" values from the "genres", "runtime_minutes", and "original_title" field. We then sorted all movies by "runtime_minutes" (highest to lowest) and removed duplicates where the movie name and year were the same.

In [88]:
IMDB_clean2 = IMDB.dropna(subset = ['genres'])

In [89]:
IMDB_clean2 = IMDB.dropna(subset = ['runtime_minutes'], how = 'any')

In [90]:
IMDB_clean2 = IMDB.dropna(subset = ['original_title'])

In [91]:
IMDB_clean2.sort_values(by='runtime_minutes', ascending = False, inplace=True)
IMDB_clean2

<ipython-input-91-7ea3f4965137>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IMDB_clean2.sort_values(by='runtime_minutes', ascending = False, inplace=True)


,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
132389,tt8273150,Logistics,Logistics,2012,51420.0,Documentary
44840,tt2659636,Modern Times Forever,Modern Times Forever,2011,14400.0,Documentary
123467,tt7492094,Nari,Nari,2017,6017.0,Documentary
87264,tt5068890,Hunger!,Hunger!,2015,6000.0,"Documentary,Drama"
88717,tt5136218,London EC1,London EC1,2015,5460.0,"Comedy,Drama,Mystery"
...,...,...,...,...,...,...
146131,tt9915790,Bobbyr Bondhura,Bobbyr Bondhura,2019,NaN,Family
146138,tt9916428,The Secret of China,The Secret of China,2019,NaN,"Adventure,History,War"
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy


In [39]:
IMDB_clean2 = IMDB_clean2.drop_duplicates(subset = ['original_title','start_year'], keep = 'first')
IMDB_clean2

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
132389,tt8273150,Logistics,Logistics,2012,51420.0,Documentary
44840,tt2659636,Modern Times Forever,Modern Times Forever,2011,14400.0,Documentary
123467,tt7492094,Nari,Nari,2017,6017.0,Documentary
87264,tt5068890,Hunger!,Hunger!,2015,6000.0,"Documentary,Drama"
88717,tt5136218,London EC1,London EC1,2015,5460.0,"Comedy,Drama,Mystery"
...,...,...,...,...,...,...
146131,tt9915790,Bobbyr Bondhura,Bobbyr Bondhura,2019,NaN,Family
146138,tt9916428,The Secret of China,The Secret of China,2019,NaN,"Adventure,History,War"
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy


We are now merging the IMDB Clean2 and IMDB Ratings using the movie ID field. Then we will merge that dataframe with the TN_clean file on two conditions - movie name and start year. In order to apply and merge on the start year condition, we will first need to create a column "release year" within the TN_clean dataframe.

In [40]:
IMDB_Merged = pd.merge(IMDB_clean2,IMDB_ratings,on='movie_id', how = 'left')
IMDB_Merged

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt8273150,Logistics,Logistics,2012,51420.0,Documentary,5.0,17.0
1,tt2659636,Modern Times Forever,Modern Times Forever,2011,14400.0,Documentary,6.2,69.0
2,tt7492094,Nari,Nari,2017,6017.0,Documentary,NaN,NaN
3,tt5068890,Hunger!,Hunger!,2015,6000.0,"Documentary,Drama",NaN,NaN
4,tt5136218,London EC1,London EC1,2015,5460.0,"Comedy,Drama,Mystery",NaN,NaN
...,...,...,...,...,...,...,...,...
144454,tt9915790,Bobbyr Bondhura,Bobbyr Bondhura,2019,NaN,Family,NaN,NaN
144455,tt9916428,The Secret of China,The Secret of China,2019,NaN,"Adventure,History,War",NaN,NaN
144456,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary,NaN,NaN
144457,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy,NaN,NaN


In [95]:
IMDB_Merged['averagerating'].value_counts().sum()

73260

In [41]:
TN_clean['release_year'] = pd.DatetimeIndex(TN_DF['release_date']).year
TN_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   release_date       5782 non-null   datetime64[ns]
 1   movie              5782 non-null   object        
 2   production_budget  5782 non-null   int64         
 3   domestic_gross     5782 non-null   int64         
 4   worldwide_gross    5782 non-null   int64         
 5   Net Income         5782 non-null   int64         
 6   release_year       5782 non-null   int64         
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 316.3+ KB


In [96]:
Movies_DF = pd.merge(TN_clean, IMDB_Merged, left_on = ["movie","release_year"], right_on = ["original_title","start_year"], how = "inner")
Movies_DF.sort_values(by='Net Income', ascending= False)

,release_date,movie,production_budget,domestic_gross,worldwide_gross,Net Income,release_year,Net_Income_Ratio,ROI,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
3,2018-04-27,Avengers: Infinity War,300000000,678815482,2048134200,1748134200,2018,0.853525,5.827114,tt4154756,Avengers: Infinity War,Avengers: Infinity War,2018,149.0,"Action,Adventure,Sci-Fi",8.5,670926.0
23,2015-06-12,Jurassic World,215000000,652270625,1648854864,1433854864,2015,0.869606,6.669092,tt0369610,Jurassic World,Jurassic World,2015,124.0,"Action,Adventure,Sci-Fi",7.0,539338.0
18,2012-05-04,The Avengers,225000000,623279547,1517935897,1292935897,2012,0.851772,5.746382,tt0848228,The Avengers,The Avengers,2012,143.0,"Action,Adventure,Sci-Fi",8.1,1183655.0
27,2018-02-16,Black Panther,200000000,700059566,1348258224,1148258224,2018,0.851660,5.741291,tt1825683,Black Panther,Black Panther,2018,134.0,"Action,Adventure,Sci-Fi",7.3,516148.0
72,2018-06-22,Jurassic World: Fallen Kingdom,170000000,417719760,1305772799,1135772799,2018,0.869809,6.681016,tt4881806,Jurassic World: Fallen Kingdom,Jurassic World: Fallen Kingdom,2018,128.0,"Action,Adventure,Sci-Fi",6.2,219125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,2020-02-21,Call of the Wild,82000000,0,0,-82000000,2020,-inf,-1.000000,tt7504726,Call of the Wild,Call of the Wild,2020,NaN,"Adventure,Animation,Family",NaN,NaN
241,2017-12-13,Bright,90000000,0,0,-90000000,2017,-inf,-1.000000,tt5519340,Bright,Bright,2017,117.0,"Action,Crime,Fantasy",6.4,147834.0
190,2019-06-14,Men in Black: International,110000000,3100000,3100000,-106900000,2019,-34.483871,-0.971818,tt2283336,Men in Black: International,Men in Black: International,2019,115.0,"Action,Adventure,Comedy",NaN,NaN
118,2011-03-11,Mars Needs Moms,150000000,21392758,39549758,-110450242,2011,-2.792691,-0.736335,tt1305591,Mars Needs Moms,Mars Needs Moms,2011,88.0,"Adventure,Animation,Family",5.4,19823.0


In [133]:
Movies_DF_Filtered = Movies_DF[ (Movies_DF['production_budget'] > 10000000) & (Movies_DF['domestic_gross'] > 0) & (Movies_DF['release_year'] > 2012)]
Movies_DF_Filtered

,release_date,movie,production_budget,domestic_gross,worldwide_gross,Net Income,release_year,Net_Income_Ratio,ROI,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
1,2019-06-07,Dark Phoenix,350000000,42762350,149762350,-200237650,2019,-1.337036,-0.572108,tt6565702,Dark Phoenix,Dark Phoenix,2019,113.0,"Action,Adventure,Sci-Fi",6.0,24451.0
2,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963,1072413963,2015,0.764364,3.243841,tt2395427,Avengers: Age of Ultron,Avengers: Age of Ultron,2015,141.0,"Action,Adventure,Sci-Fi",7.3,665594.0
3,2018-04-27,Avengers: Infinity War,300000000,678815482,2048134200,1748134200,2018,0.853525,5.827114,tt4154756,Avengers: Infinity War,Avengers: Infinity War,2018,149.0,"Action,Adventure,Sci-Fi",8.5,670926.0
4,2017-11-17,Justice League,300000000,229024295,655945209,355945209,2017,0.542645,1.186484,tt0974015,Justice League,Justice League,2017,120.0,"Action,Adventure,Fantasy",6.5,329135.0
5,2015-11-06,Spectre,300000000,200074175,879620923,579620923,2015,0.658944,1.932070,tt2379713,Spectre,Spectre,2015,148.0,"Action,Adventure,Thriller",6.8,352504.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,2015-04-01,Woman in Gold,11000000,33307793,58586407,47586407,2015,0.812243,4.326037,tt2404425,Woman in Gold,Woman in Gold,2015,109.0,"Biography,Drama,History",7.3,48501.0
1003,2013-12-06,Inside Llewyn Davis,11000000,13248209,32943247,21943247,2013,0.666092,1.994841,tt2042568,Inside Llewyn Davis,Inside Llewyn Davis,2013,104.0,"Drama,Music",7.5,123759.0
1004,2013-05-10,Peeples,11000000,9177065,9307166,-1692834,2013,-0.181885,-0.153894,tt1699755,Peeples,Peeples,2013,95.0,"Comedy,Romance",5.3,4452.0
1005,2016-07-06,Sultan,11000000,5599781,72989781,61989781,2016,0.849294,5.635435,tt4832640,Sultan,Sultan,2016,170.0,"Action,Drama,Sport",7.1,33371.0


In [130]:
Movies_DF['release_year'].value_counts()

2015    188
2014    179
2010    177
2011    174
2013    165
2012    160
2016    156
2018    118
2017    112
2019     41
2020      1
Name: release_year, dtype: int64

In [112]:
Movie_Genre_ROI = Movies_DF.groupby('genres')['Net Income'].mean()
Movie_Genre_ROI.sort_values(ascending=False)

genres
Family,Fantasy,Musical        1.099200e+09
Adventure,Drama,Sci-Fi        5.243254e+08
Action,Adventure,Sci-Fi       4.722965e+08
Adventure,Fantasy             4.695440e+08
Action,Adventure,Animation    3.595655e+08
                                  ...     
Action,Fantasy,Western       -3.348568e+07
Crime,Drama,History          -4.199578e+07
Adventure,Drama,Mystery      -4.980000e+07
Action,Family,Fantasy        -6.953398e+07
Action,Crime,Fantasy         -9.000000e+07
Name: Net Income, Length: 227, dtype: float64

In [140]:
Movies_DF_Filtered['genres'].value_counts(ascending=True).head(40)


Action,Crime                1
Fantasy                     1
Biography,Drama,Romance     1
Action,Comedy,Fantasy       1
Biography,Drama,Family      1
Comedy,Family,Fantasy       1
Music                       1
Sci-Fi,Thriller             1
Comedy,Drama,Mystery        1
Drama,Music,Thriller        1
Crime,Mystery,Thriller      1
Action,Mystery,Sci-Fi       1
Adventure,Drama,Horror      1
Action,Crime,Sport          1
Comedy,Drama,Family         1
Adventure,Drama,Romance     1
Crime,Horror,Mystery        1
Comedy,Drama,Sport          1
Comedy,Fantasy,Horror       1
Fantasy,Horror,Mystery      1
Comedy,Horror,Romance       1
Action,Crime,Mystery        1
Drama,Family,Sport          1
Comedy,Family               1
Drama,Horror                1
Drama,Fantasy,Music         1
Biography,Drama,War         1
Adventure,Comedy,Romance    1
Fantasy,Horror,Thriller     1
Adventure,Drama,Fantasy     1
Action,Comedy               1
Crime,Drama,History         1
Comedy,Sport                1
Drama,War 